In [ ]:
# T

In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from datetime import timezone, datetime
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
imputedMeasurementsDf = pd.read_csv('../../../../../../Data/ProcessedData/KnnImputed/measurementsImputedByKnn.csv')
measurementsDf = pd.read_csv('../../../../../../Data/RawData/rawMeasurementsFilteredBySensors.csv')

In [3]:
dateTimeDf = pd.DataFrame(measurementsDf['time'])

In [4]:
def iso8601ToDatetime():
    for i in range(len(dateTimeDf['time'])):
        timeContainer  = dateTimeDf['time'][i].replace('.000','')
        timeContainer = datetime.strptime(timeContainer, "%Y-%m-%dT%H:%M:%SZ")
        timeContainer = int(round(timeContainer.timestamp()))
        timeContainer = datetime.fromtimestamp(timeContainer)
        dateTimeDf['time'][i] = timeContainer

In [5]:
%%time

iso8601ToDatetime()

CPU times: total: 14.2 s
Wall time: 14.7 s


In [6]:
measurementsDf = dateTimeDf.join(imputedMeasurementsDf.drop(['day','month','hour'],axis = 1))

In [7]:
measurementsDf

,time,T,Rn,H,P,CO2,sensor_id
0,2020-07-14 14:00:00,31.300000,1001.448424,44.475000,101.565847,567.750000,D0001
1,2020-07-14 15:00:00,31.333333,588.154974,45.505556,101.632854,636.361111,D0001
2,2020-07-14 16:00:00,31.451429,0.100000,47.145714,101.539128,734.000000,D0001
3,2020-07-14 17:00:00,31.161765,1042.843803,36.000000,101.501998,461.529412,D0001
4,2020-07-14 18:00:00,31.008333,1585.713833,28.436111,101.567130,400.000000,D0001
...,...,...,...,...,...,...,...
115629,2021-12-31 19:00:00,29.792673,616.640000,50.156364,102.653100,97.200000,D009
115630,2021-12-31 20:00:00,29.951673,541.380000,48.047273,102.273400,97.800000,D009
115631,2021-12-31 21:00:00,30.329218,544.167273,53.847273,102.199727,97.181818,D009
115632,2021-12-31 22:00:00,29.511200,590.666000,44.097576,102.140000,98.100000,D009


In [8]:
measurementsDf.index =measurementsDf['time']

In [9]:
temperatureDf = measurementsDf['T']

In [10]:
temperatureDf

time
2020-07-14 14:00:00    31.300000
2020-07-14 15:00:00    31.333333
2020-07-14 16:00:00    31.451429
2020-07-14 17:00:00    31.161765
2020-07-14 18:00:00    31.008333
                         ...    
2021-12-31 19:00:00    29.792673
2021-12-31 20:00:00    29.951673
2021-12-31 21:00:00    30.329218
2021-12-31 22:00:00    29.511200
2021-12-31 23:00:00    30.079564
Name: T, Length: 115634, dtype: float64

In [11]:
#temperatureDf.plot()

In [12]:
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
        
    return np.array(X), np.array(y)

In [13]:
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(temperatureDf, WINDOW_SIZE)
X1.shape, y1.shape

((115629, 5, 1), (115629,))

In [14]:
X_train1, y_train1 = X1[:80000], y1[:80000]
X_val1, y_val1 = X1[80000:85000], y1[80000:85000]
X_test1, y_test1 = X1[85000:], y1[85000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((80000, 5, 1), (80000,), (5000, 5, 1), (5000,), (30629, 5, 1), (30629,))

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()

model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dense (Dense)                (None, 8)                 520       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 17,425
Trainable params: 17,425
Non-trainable params: 0
_________________________________________________________________


In [19]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [20]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])

Epoch 1/10
2500/2500 [==============================] - 22s 8ms/step - loss: 148.4951 - root_mean_squared_error: 12.1859 - val_loss: 0.5921 - val_root_mean_squared_error: 0.7695


INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


Epoch 2/10
2500/2500 [==============================] - 17s 7ms/step - loss: 1.4499 - root_mean_squared_error: 1.2041 - val_loss: 0.4543 - val_root_mean_squared_error: 0.6740


INFO:tensorflow:Assets written to: model1\assets


...

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


Epoch 6/10
2500/2500 [==============================] - 17s 7ms/step - loss: 1.0571 - root_mean_squared_error: 1.0282 - val_loss: 0.4034 - val_root_mean_squared_error: 0.63511.06
Epoch 7/10
2500/2500 [==============================] - 19s 7ms/step - loss: 1.0536 - root_mean_squared_error: 1.0265 - val_loss: 0.4023 - val_root_mean_squared_error: 0.6343
Epoch 8/10
2500/2500 [==============================] - 17s 7ms/step - loss: 1.0499 - root_mean_squared_error: 1.0247 - val_loss: 0.4004 - val_root_mean_squared_error: 0.6327: 1s - loss: 1.0417 - root_mean_squared_erro - ETA: 1s - loss: 1.0449 - root_mean_sq - ETA: 0s - loss: 1.0373 - root_mean_squared_error: 
Epoch 9/10
2500/2500 [==============================] - 17s 7ms/step - loss: 1.0471 - root_mean_squared_error: 1.0233 - val_loss: 0.3993 - val_root_mean_squared_error: 0.6319oot_mean_squared_ - ETA: 2s - loss: 1.0383 - root_mean_squared_error:  - ETA: 2s - loss: 1.0427 - root_mean_squared_error: - ETA: 2s - loss: 1.0392 - root_mean_

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


In [15]:
from tensorflow.keras.models import load_model
model = load_model('LSTM_model/')

In [16]:
train_predictions = model.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

,Train Predictions,Actuals
0,31.120104,30.647222
1,30.878284,29.891429
2,30.390657,29.118919
3,29.762238,28.200000
4,28.935419,28.280556
...,...,...
79995,26.000505,26.082500
79996,26.033413,25.911667
79997,25.904177,26.011818
79998,25.956099,25.944167


In [17]:
#import matplotlib.pyplot as plt
#plt.plot(train_results['Train Predictions'][50:100])
#plt.plot(train_results['Actuals'][50:100])

In [19]:
val_predictions = model.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})

,Val Predictions,Actuals
0,25.797438,25.822000
1,25.790070,25.745000
2,25.722664,25.707500
3,25.678833,25.654545
4,25.627695,25.607500
...,...,...
4995,25.632406,25.659167
4996,25.662062,25.821667
4997,25.753841,26.438889
4998,26.214628,26.259091


In [21]:
val_results.head(20)

,Val Predictions,Actuals
0,25.797438,25.822000
1,25.790070,25.745000
2,25.722664,25.707500
3,25.678833,25.654545
4,25.627695,25.607500
5,25.579926,25.837500
6,25.738695,26.072500
7,25.935349,26.224545
8,26.089977,26.718333
9,26.500008,27.246667
